In [28]:
%load_ext autoreload
%autoreload 1



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import pandas as pd
import os
from mimic3benchmarks.mimic3models.preprocessing import Discretizer
import numpy as np
from tqdm import tqdm

dir_path  = '/data1/r10user2/EHR_dataset/mimiciv_benchmark/survival_prediction'

train_path = os.path.join(dir_path , 'train_listfile.csv')
test_path = os.path.join(dir_path , 'test_listfile.csv')

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

x_train = []
del_data = 0
for patient_path in tqdm(train_df['stay']):
    patient_path = os.path.join(dir_path, 'train', patient_path)
    patient = pd.read_csv(patient_path)
    patient = patient.fillna('')
    data_processor = Discretizer(impute_strategy='normal_value')
    data = data_processor.transform(np.array(patient))[0]
    if data.shape[0]  < 60:
        num_pad = 60 - data.shape[0]
        #print(num_pad)
        last_data = data[-1].repeat(num_pad,axis=0).reshape(-1,76)
        #print(last_data.shape)
        data = np.concatenate([data, last_data])

    if data.shape[0] == 60:
        x_train.append(data)
    else:
        del_data += 1
       
print(del_data) 
    

100%|██████████| 18064/18064 [07:46<00:00, 38.72it/s]

0


In [10]:
x_train = np.array(x_train)
t_train = np.array(train_df['survival_time'])
e_train = np.array(train_df['censor'])
np.save("x_train.npy", x_train)
np.save("t_train.npy", t_train)
np.save("e_train.npy", e_train)

In [4]:
t_train

array([ 47.38      , 119.7       ,  22.63472222, ...,  48.59055556,
        32.99166667,  16.94916667])

In [11]:
x_test = []
del_data = 0
for patient_path in tqdm(test_df['stay']):
    patient_path = os.path.join(dir_path, 'test', patient_path)
    patient = pd.read_csv(patient_path)
    patient = patient.fillna('')
    data_processor = Discretizer(impute_strategy='normal_value')
    data = data_processor.transform(np.array(patient))[0]
    if data.shape[0]  < 60:
        num_pad = 60 - data.shape[0]
        #print(num_pad)
        last_data = data[-1].repeat(num_pad,axis=0).reshape(-1,76)
        #print(last_data.shape)
        data = np.concatenate([data, last_data])

    if data.shape[0] == 60:
        x_test.append(data)
    else:
        del_data += 1
       
print(del_data) 

  0%|          | 0/4972 [00:00<?, ?it/s]

100%|██████████| 4972/4972 [01:19<00:00, 62.85it/s]


0


In [12]:
x_test = np.array(x_test)
t_test = np.array(test_df['survival_time'])
e_test = np.array(test_df['censor'])
np.save("x_test.npy", x_test)
np.save("t_test.npy", t_test)
np.save("e_test.npy", e_test)

In [6]:
print(x_train.shape)
print(t_train.shape)

(18064, 60, 76)
(18064,)


In [25]:
import torch.nn as nn
import torch.nn.functional as F
import torch

def LSTM_transform(x):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    x = torch.from_numpy(x)
    x = x.to(torch.float32)
    x = x.to(device)
    embedding = nn.LSTM(input_size = 76, hidden_size = 76, num_layers = 2, batch_first = True).to(device)
    x_rep = embedding(x)
    return x_rep


In [26]:
x_rep = LSTM_transform(x_train)
x_rep[0].astype

OutOfMemoryError: CUDA out of memory. Tried to allocate 3.38 GiB (GPU 0; 23.70 GiB total capacity; 19.56 GiB already allocated; 1.25 GiB free; 20.09 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [9]:
import numpy as np
x_train = np.load('x_train.npy')[:500]
t_train = np.load('t_train.npy')[:500]
e_train = np.load('e_train.npy')

In [11]:
print(e_train.sum()/len(e_train))

0.8774911426040743


In [2]:
np.isnan(x_train).sum()

0

In [3]:
np.isnan(t_train).sum()

0

In [4]:
np.isnan(e_train).sum()

0

In [5]:
models = []

from auton_survival.models.dsm import DeepRecurrentSurvivalMachines
from auton_survival.models.dsm import DeepSurvivalMachines

# model = DeepRecurrentSurvivalMachines(k=3,
#                distribution='LogNormal',
#                layers=10)
model = DeepSurvivalMachines(
    k=4,
    distribution="LogNormal",
    layers=[76]
)
# The fit method is called to train the model
model.fit(x_train, t_train, e_train, iters=1, learning_rate=1e-5)








/home/r10user10/Documents/anaconda3/envs/python310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 3/10000 [00:00<07:48, 21.34it/s]


is f nana tensor(1)
is s nana tensor(1)
0
tensor(True)
is f nana tensor(0)
is s nana tensor(0)
is f nana tensor(1)
is s nana tensor(1)
0
tensor(True)
is f nana tensor(0)
is s nana tensor(0)
is f nana tensor(1)
is s nana tensor(1)
0
tensor(True)
is f nana tensor(0)
is s nana tensor(0)
is f nana tensor(1)
is s nana tensor(1)
0
tensor(True)
is f nana tensor(0)
is s nana tensor(0)


100%|██████████| 1/1 [00:56<00:00, 56.73s/it]


In [6]:
trained_weights = model.trained_weights
print(trained_weights.shape)
print(np.isnan(trained_weights).sum())

(1, 425, 4)
500


In [7]:
horizons = [0.25, 0.5, 0.75]
times = np.quantile(t[e == 1], horizons).tolist()
out_risk = model.predict_risk(x_test, times)
out_survival = model.predict_survival(x_test, times)

from sksurv.metrics import concordance_index_ipcw, brier_score, cumulative_dynamic_auc

cis = []
brs = []

et_train = np.array([(e_train[i], t_train[i]) for i in range(len(e_train))],
                 dtype = [('e', bool), ('t', float)])
et_test = np.array([(e_test[i], t_test[i]) for i in range(len(e_test))],
                 dtype = [('e', bool), ('t', float)])
# et_val = np.array([(e_val[i], t_val[i]) for i in range(len(e_val))],
#                  dtype = [('e', bool), ('t', float)])

for i, _ in enumerate(times):
    cis.append(concordance_index_ipcw(et_train, et_test, out_risk[:, i], times[i])[0])
brs.append(brier_score(et_train, et_test, out_survival, times)[1])
roc_auc = []
for i, _ in enumerate(times):
    roc_auc.append(cumulative_dynamic_auc(et_train, et_test, out_risk[:, i], times[i])[0])
for horizon in enumerate(horizons):
    print(f"For {horizon[1]} quantile")
    print("TD Concordance Index:", cis[horizon[0]])
    print("Brier Score:", brs[0][horizon[0]])
    print("ROC AUC ", roc_auc[horizon[0]][0], "\n")

NameError: name 't' is not defined

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

figs, axes = plt.subplots(1, 3, figsize=(15, 5))
iter_idx = [97, 98, 99]

for idx in range(3):
    sns.kdeplot(trained_weights[iter_idx[idx]][:, 0], fill=True, ax=axes[idx])
    sns.kdeplot(trained_weights[iter_idx[idx]][:, 1], fill=True, ax=axes[idx])
    sns.kdeplot(trained_weights[iter_idx[idx]][:, 2], fill=True, ax=axes[idx])
    sns.kdeplot(trained_weights[iter_idx[idx]][:, 3], fill=True, ax=axes[idx])
    axes[idx].set_title(f'Iter {iter_idx[idx]}')
    axes[idx].set_xlim(0.15, 0.4)